# predict on sequence

this is predicting on the in vivo stuff

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

import sys
sys.path.insert(0,'..')

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [ ]:
# WE TRAIN A MODEL 
from optimization.optimize_forest import quickloaddata as load
import simushape_nostruct as sn
import simushape as ss
data = load(dataset='16')

#print len(data['Group1T'][0])
#print ss.getXY(data,['Group1T'])

In [ ]:
import xgboost as xgb
reg = xgb.XGBRegressor(**{'reg_alpha': 0.8500639221640397, 'learning_rate': 0.01766263588148119, 'max_delta_step': 1, 'min_child_weight': 3, 'n_estimators': 127, 'reg_lambda': 0.70393814277000044, 'max_depth': 14, 'gamma':0, 'booster': 'dart'})
model = ss.make_model(data, data.keys(),model=reg)


In [ ]:
# WE PREDICT

import simushape as ss
import simushape_nostruct as sn
import notebook_helper as helper
from IPython.display import HTML, display
import tabulate
test_fasta = ss.read_fasta('../data/eval_rrna/galaxy31.fasta')
test_react = ss.read_react('../data/eval_rrna/galaxy31.react')
   
import rnafold as rf


def predict(model,test_fasta):
    res=[]
    for k,v in test_fasta:
        if len(v) < 130:
            a,b = sn.predict_and_scale(model,v,150)
            res.append((k,v,a,b))
    return res
        
def draw_res(res):
    for name, seq, react, react2 in res:
        print name
        INVIVO=test_react[name]
        struc = rf.fold(seq,react) # we use this for outpu
        struc2 = rf.fold(seq,INVIVO)
        structhighlight = [ 0 if b=='.' and a!='.' else .5 for a,b in zip(struc,struc2)    ] # b is invivo 
        print structhighlight
        helper.draw_seq_rea(seq,[INVIVO,react,react2,structhighlight],struc)

        y,x = ss.mask(react,INVIVO)
        z,x = ss.mask(react2,INVIVO)
        table= [x,y,z]
        #table = zip(*table)
        display(HTML(tabulate.tabulate(table, tablefmt='html')))  
        
   
        print 'norm ',struc
        print 'pred',struc2
        
        


#print '\t'.join( map(str,(list(res[0][2]))))
#print ss.format_shape('asd',res[0][2])
#print test_fasta


#print sn.shape(test_fasta[0][1])



In [ ]:

res = predict(model,test_fasta)

draw_res(res)


# DRAW FUNCTION FOR THE SHAPE DATA

In [ ]:
print test_fasta

In [ ]:
def debug_structures(fasta,react):
    for name,seq in fasta:
        react_= react[name]
        struc = rf.fold(seq,react_) # we use this for outpu
        mep = [x/7.0 if x!=None else None for x in np.array(react_)]
        
        struc3 = rf.fold(seq,mep) # we use this for outpu
      
        print struc
        print struc3
        print
        
debug_structures(test_fasta[:5], test_react)